##### Copyright 2023 Google LLC

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-pro' # @param {isTemplate: true}
contents_b64 = 'W10=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Upload the video and print a confirmation.
video_file_name = "/content/drive/MyDrive/微观经济学/MIT微观经济学ProductionTHeory.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/u6ea2vjh3612
..........

In [5]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

# Create the prompt.
prompt = """角色设定: 你是一个学生，正在学习刑法课，想要做到能够清晰、准确地理解法律文本，并能结合实际案例进行分析。
任务描述: 请观看我下面给出的刑法总论的关于贿赂犯罪的课程视频。在观看完视频后，请回答下列问题。确保你的回答详细、准确，并结合视频中的相关信息来支持你的答案。
"""
contents = [prompt]

print("Making LLM inference request...")
response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)
print("Successfully generating reponse:")
display(Markdown(response.text))

Making LLM inference request...
Successfully generating reponse:


请提供课程视频。我没有观看视频的功能，无法根据视频内容回答你的问题。请你提供视频的文字稿或者总结视频中关于贿赂罪的关键信息，例如：

* 贿赂罪的定义和构成要件
* 不同类型的贿赂罪（例如：受贿罪、行贿罪、单位行贿罪等等）及其区别
* 贿赂罪的认定标准，例如“财物”的范围、“职务行为”的解释
* 相关案例分析，包括案情描述、法院判决以及判决理由
* 免责事由和从轻、减轻处罚情节

你提供的信息越详细，我就能更好地帮助你理解贿赂罪的相关知识，并进行案例分析。


## Call `generate_content`

In [6]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

# Create the prompt.
prompt = """
角色设定: 你是一个学生，正在学习刑法课，想要做到能够清晰、准确地理解法律文本，并能结合实际案例进行分析。
任务描述: 请观看我下面给出的刑法总论的关于贿赂犯罪的课程视频。在观看完视频后，请回答下列问题。确保你的回答详细、准确，并结合视频中的相关信息来支持你的答案。
问题列表:
1.	受贿罪的本质是什么？
2.	受贿罪的既遂标准是什么？
3.	性贿赂在什么情况下构成受贿罪？
4.	国家工作人员收受财物后，什么情况下不构成受贿罪？
5.	受贿罪的五种行为方式是什么？
6.	索贿和诈骗罪的区别是什么？
7.	单纯受贿的成立条件是什么？
8.	斡旋受贿的构成要件是什么？
9.	事后受贿如何认定？
10.	行贿罪的构成要件是什么？
11.	某医院院长收受医药代表提供的旅游机会，价值10万元，但院长并未为该医药代表谋取任何利益。院长是否构成受贿罪？
12.	某公司为获得政府项目，向主管官员赠送了一幅名画，价值50万元。事后，该公司获得了该项目。这幅名画如何认定？
13.	某官员的妻子收受了商人20万元现金，并承诺让其丈夫在项目审批上给予关照。该官员对妻子收钱的事情并不知情。妻子和商人分别构成什么罪？
14.	某官员在退休后收受了企业50万元，感谢其在职期间的关照。这构成事后受贿吗？如何认定？
15.	某企业在与国企进行经济往来时，按照行业惯例，给予国企负责人一定的回扣。这是否构成行贿罪？
16.	某公司为了获得一项工程，向官员行贿10万元，但该官员并未承诺任何事情，也未实际为该公司谋取利益。这构成行贿罪吗？
17.	李某为帮助朋友获得银行贷款，向银行工作人员行贿5万元。但由于该朋友的资质不符合贷款条件，最终未能获得贷款。李某的行为如何认定？
18.	某领导的司机经常收受一些企业送的购物卡和土特产，价值数千元，并告知领导是谁送的。领导未表示反对，也未让司机退还。领导是否构成受贿罪？
19.	张某的儿子想参军，张某向负责征兵的官员送了2万元。但其儿子因身体原因未能通过体检。张某的行为如何认定？
20.	某官员的秘书利用官员的影响力，收受企业30万元，承诺为企业在项目审批上提供帮助。官员对秘书的行为并不知情。秘书的行为如何认定？
21.	丙为某税务局局长，刘某因企业漏税被查，找丙疏通关系，并交给丙2万元现金，被丙婉拒。两人寒暄一二，丙问刘某是否能帮其安插情妇王女到企业工作，刘某答应。次日刘某聘任并无会计资格的王女为企业财务副总监，王女从未上班，但刘某按月支付万元工资。丙构成受贿罪吗？
22.	张三对求他在办公楼租售中“行方便”的吴某谎称，自己的外甥准备出国留学。吴某于是将20万元打入张三指定的银行卡账户。张三该当何罪？
23.	你有印象深刻的贪腐案件吗？
24.	张三向某高校招生人员刘某送2万元，希望刘某在招生时录取自己的女儿。遭到刘某婉拒后，张三又找到刘某的妻子，其妻子私自把钱收下了。张三是否构成行贿罪？
25.	某建筑公司（不具有建设隧道的资质）老板张三是王某的同学，张三找到王某，希望王某向其父（某城建局局长）说说情，把隧道工程发包给自己，并给了王某价值50万元去韩国整容的贵宾卡及往返机票。王某向父亲说了此事，并出示了机票和贵宾卡。其父将工程发包给了张三。王某是否构成利用影响力受贿罪？
请你基于视频内容，开始回答问题，回答格式是先列出问题，再给出答案。对于无法从视频中推断出答案的问题，请回答视频并未提及有关的知识点，无法作答。
"""
contents = [video_file, prompt]

print("Making LLM inference request...")
response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)
print("Successfully generating reponse:")
display(Markdown(response.text))

Making LLM inference request...
Successfully generating reponse:


Sure, here are the answers to your questions, based on the MIT OpenCourseWare video "Lecture 5: Production Theory" for the course "Principles of Microeconomics (14.01)," Fall 2018.

**Knowledge Questions (10):**

1. **What is a production function?**
A production function describes the relationship between the inputs a firm uses and the quantity of output it produces. It shows the maximum amount of output that can be produced from a given set of inputs, given the available technology.  Mathematically, it can be represented as q = f(L,K), where q is the quantity of output, L is labor, and K is capital.

2. **What is the firm's primary objective in producer theory?**
The firm's primary objective in producer theory is to maximize profits (Π), which is defined as the difference between total revenue (R) and total cost (C):  Max Π = R - C.

3. **Define "factors of production."**
Factors of production are the inputs used by firms to produce goods and services.  The video focuses on two primary factors: labor (L) and capital (K).  Other factors, such as land and raw materials, are sometimes considered as well.

4. **Distinguish between variable and fixed inputs.**
Variable inputs are those that can be changed in the short run, such as the number of workers employed.  Fixed inputs are those that cannot be changed in the short run, like the size of a factory or the number of machines.

5. **How are the short run and long run defined in producer theory?**
The short run is the period of time during which at least one input is fixed. The long run is the period of time during which all inputs are variable.

6. **What is an isoquant?**
An isoquant is a curve that shows all the different combinations of inputs (L and K) that can produce the same level of output (q).  It's like an indifference curve for producers.

7. **What does the marginal rate of technical substitution (MRTS) represent?**
The MRTS represents the rate at which a firm can substitute one input for another (e.g., capital for labor) while holding output constant.  It is the slope of the isoquant.

8. **What is the mathematical relationship between MRTS and marginal products of labor (MPL) and capital (MPK)?**
MRTS = -ΔK/ΔL = MPL/MPK.  This means the MRTS is equal to the ratio of the marginal product of labor to the marginal product of capital.

9. **What does the term "returns to scale" refer to?**
Returns to scale describes how output changes when all inputs are increased proportionately.  There are three types: increasing returns to scale (output increases more than proportionally), constant returns to scale (output increases proportionally), and decreasing returns to scale (output increases less than proportionally).

10. **Give an example of an industry with potentially decreasing returns to scale.**
The video does not offer a specific example, though the speaker mentions industries that are dependent on fixed natural resources might exhibit this. The speaker presents an example with tobacco where output increases less than proportionally to increases in the inputs.



**Application and Analysis Questions (10):**

11. **A software company uses programmers (L) and computers (K) to produce lines of code (q) according to the production function q = 10L^0.5 K^0.5. Does this production function exhibit increasing, decreasing, or constant returns to scale? Explain your answer.**
This production function exhibits constant returns to scale.  If you double both L and K, output doubles. For example: q = 10(2L)^0.5(2K)^0.5 = 10*2^0.5*L^0.5*2^0.5*K^0.5 = 2*10L^0.5K^0.5 = 2q.

12. **Suppose a firm’s production function is q = L + K. Draw the isoquants for q = 10, q = 20, and q = 30. What does the shape of these isoquants tell you about the relationship between labor and capital in this production process?**
The isoquants would be straight lines with a slope of -1.  This indicates that labor and capital are perfectly substitutable in this production process; one unit of labor can always be replaced by one unit of capital, and vice versa.

13. **A furniture maker experiences increasing returns to scale initially, followed by decreasing returns to scale as output grows. Explain why this might occur, relating your answer to specialization and coordination costs.**
Initially, increasing returns might occur due to specialization.  As the firm grows, workers can specialize in specific tasks, leading to higher productivity. However, as the firm gets much larger, coordination costs can increase.  Managing a large number of specialized workers can become complex and inefficient, leading to decreasing returns.

14. **Consider a Leontief production function for making bicycles, where wheels (W) and frames (F) are the inputs: q = min(W, F). What are the implications of this production function for a bicycle manufacturer's input choices?**
This production function implies that wheels and frames are perfect complements.  The manufacturer needs exactly one wheel and one frame for each bicycle.  Having more of one input without a corresponding increase in the other will not increase output. They must be used in fixed proportions.

15. **The lecture discusses the historical trend of productivity growth in the US. If productivity growth remains low, what are the implications for future increases in the standard of living? What policies could potentially stimulate productivity growth?**
Low productivity growth leads to slower growth in the standard of living, meaning wages and overall economic well-being will improve more slowly over time.  Policies to stimulate productivity growth include investment in education, research and development, infrastructure, and policies that promote competition and innovation.

16. **Discuss the trade-off between increased consumption and increased leisure that societies face as productivity grows. Use examples to illustrate how different countries have made different choices in this regard.**
As productivity grows, societies can choose to produce more goods and services (increased consumption) or to work less and enjoy more leisure time.  The US, for example, has generally favored increased consumption over increased leisure compared to many European countries, which often have higher taxes, stronger social safety nets, and more vacation time.

17. **Why is innovation crucial for long-run economic growth, particularly given the concept of diminishing marginal product?**
Diminishing marginal product implies that adding more of the same inputs (labor and capital) will eventually lead to smaller and smaller increases in output.  Innovation, by improving technology and creating new products and processes (represented by the “A” term in the production function), is crucial for shifting the production function upward and overcoming diminishing marginal product, leading to sustained economic growth.

18. **The lecture highlights that the benefits of productivity growth have not been equally distributed since the 1970s. What are some potential explanations for this unequal distribution?**
Some explanations include skill-biased technological change (technology favors highly skilled workers), globalization and increased competition (lowering wages for some), declining unionization (reducing bargaining power), and changes in government policies (tax cuts for the wealthy).

19. **Explain why Malthus' prediction of mass starvation due to limited land availability did not come to pass.**
Malthus failed to anticipate the impact of technological innovation in agriculture.  Innovations like fertilizers, pesticides, and genetically modified crops dramatically increased agricultural productivity, allowing food production to keep pace with population growth.

20. **Imagine a firm with the production function q = 2L + K. If the wage rate (w) is $10 and the rental rate of capital (r) is $5, what combination of labor and capital should the firm use to minimize the cost of producing 100 units of output?**
Since capital is cheaper per unit of output produced, the firm should use only capital to minimize costs. It will use K = 100 and L = 0.



Let me know if you have any other questions.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")